# Cell Types Project Master Notebook

In [2]:
# maybe this requires actual code

a = 12

print(a)

12


In [1]:
#Import the "Cell Types Cache" from the AllenSDK core package
from allensdk.core.cell_types_cache import CellTypesCache

#Import CellTypesApi, which will allow us to query the database.
from allensdk.api.queries.cell_types_api import CellTypesApi

# initialize cache as 'ctc' (cell types cache)
ctc = CellTypesCache(manifest_file='cell_types/manifest.json')

# import tool used to get AP spike data for specimen
from allensdk.ephys.ephys_extractor import EphysSweepFeatureExtractor

import pandas as pd
import numpy as np

# get rid of all the ugly red warnings
import warnings
warnings.filterwarnings('ignore')

The paper used mouse data, so filter to get only mouse specimen.

In [16]:
mouse_cells = ctc.get_cells(species = ['mus musculus'])
mouse_cells_df = pd.DataFrame(mouse_cells)
mouse_ids_df = mouse_cells_df[['id', 'species']].set_index('id')
mouse_ids_df

,species
id,
565871768,Mus musculus
469801138,Mus musculus
605889373,Mus musculus
485909730,Mus musculus
323865917,Mus musculus
...,...
512322162,Mus musculus
488473535,Mus musculus
467003163,Mus musculus


Get ephys features of the mouse species only

In [26]:
ephys_features = ctc.get_ephys_features()
ephys_features_df = pd.DataFrame(ephys_features)
ephys_ids_df = ephys_features_df[['id','specimen_id','rheobase_sweep_number']].set_index('specimen_id')
ephys_ids_df
ephys_ids_df

,id,rheobase_sweep_number
specimen_id,,
529878215,529951870,78
548459652,549196534,55
579978640,579986979,38
439024551,439816233,41
515188639,515191763,36
...,...,...
569955172,569957302,58
573410831,573464735,42
576285618,576324611,39


Use inner join to get specimen ids of only mouse species.

Is 'id' in mouse cells df the same as 'specimen id' in ephys data? Assuming yes, we can do the following to get the rheobase sweep number for each mouse specimen

In [31]:
mouse_species_df = mouse_ids_df.join(ephys_ids_df)
mouse_species_df

,species,id,rheobase_sweep_number
id,,,
565871768,Mus musculus,565878140,40
469801138,Mus musculus,469802615,57
605889373,Mus musculus,605903794,53
485909730,Mus musculus,485911653,33
323865917,Mus musculus,396487207,34
...,...,...,...
512322162,Mus musculus,512323887,44
488473535,Mus musculus,488474973,57
467003163,Mus musculus,469566263,45


In [35]:
def calc_spike_width(specimen_id):
    
    data_set = ctc.get_ephys_data(specimen_id)

    sweep_number = mouse_species_df.loc[specimen_id]['rheobase_sweep_number']
    sweep_data = data_set.get_sweep(sweep_number)

    index_range = sweep_data["index_range"]
    i = sweep_data["stimulus"][0:index_range[1]+1] # in A
    v = sweep_data["response"][0:index_range[1]+1] # in V
    i *= 1e12 # to pA
    v *= 1e3 # to mV

    sampling_rate = sweep_data["sampling_rate"] # in Hz
    t = np.arange(0, len(v)) * (1.0 / sampling_rate)

    sweep_ext = EphysSweepFeatureExtractor(t=t, v=v, i=i, start=1.02, end=2.02)
    sweep_ext.process_spikes()

    return round(1e3 * np.nanmean(sweep_ext.spike_feature("width")),4)

In [36]:
calc_spike_width(565871768)

0.44

Randomly sample 500 rows of mouse species df